# Training code package setup for AI Platform

It's necessary to create a Python package to hold the training code.  Here we're going to get started with that by creating a folder for the package and adding an empty `__init__.py` file.

In [1]:
import os
import tarfile
from google.cloud import storage
import env

In [2]:
ROOT_PATH = 'AI_Platform/trainig_package'
PACKAGE_FOLDER = '/trainer'

!rm -r {ROOT_PATH}
!mkdir {ROOT_PATH}
!mkdir {ROOT_PATH+PACKAGE_FOLDER}
!touch {ROOT_PATH+PACKAGE_FOLDER}/__init__.py
!ls -l {ROOT_PATH+PACKAGE_FOLDER}

total 0
-rw-r--r--  1 ikersanchez  staff  0 Feb 13 16:24 __init__.py


**Setuptools file named `setup.py`**

In [3]:
%%writefile {ROOT_PATH}/setup.py

from setuptools import find_packages
from setuptools import setup

REQUIRED_PACKAGES = ['keras==2.2.4']

setup(
    name='trainer',
    version='0.2',
    install_requires=REQUIRED_PACKAGES,
    packages=find_packages(),
    include_package_data=True,
    description='My training application package.'
)

Writing AI_Platform/trainig_package/setup.py


**Training and validation data**

The following is code to load training/evaluation data.  Write this into `util.py`.

In [4]:
%%writefile {ROOT_PATH+PACKAGE_FOLDER}/util.py
"""Utilities to download and preprocess the data."""

import tensorflow as tf
import json
from google.cloud import storage
from google.cloud.storage import blob

class Util():

    def __init__(self, path):
        self.path = path
        
        self.client = storage.Client(project='skydipper-196010')
        self.bucket = self.client.get_bucket('geo-ai')
        self.blob = self.bucket.blob(self.path)
        self.config = json.loads(self.blob.download_as_string(client=self.client).decode('utf-8'))
        
    def parse_function(self, proto):
        """The parsing function.
        Read a serialized example into the structure defined by features_dict.
        Args:
          example_proto: a serialized Example.
        Returns: 
          A dictionary of tensors, keyed by feature name.
        """
        
        # Define your tfrecord 
        features = self.config.get('in_bands') + self.config.get('out_bands')
        
        # Specify the size and shape of patches expected by the model.
        kernel_shape = [self.config.get('kernel_size'), self.config.get('kernel_size')]
        columns = [
          tf.io.FixedLenFeature(shape=kernel_shape, dtype=tf.float32) for k in features
        ]
        features_dict = dict(zip(features, columns))
        
        # Load one example
        parsed_features = tf.io.parse_single_example(proto, features_dict)
    
        # Convert a dictionary of tensors to a tuple of (inputs, outputs)
        inputs_list = [parsed_features.get(key) for key in features]
        stacked = tf.stack(inputs_list, axis=0)
        
        # Convert the tensors into a stack in HWC shape
        stacked = tf.transpose(stacked, [1, 2, 0])
        
        return stacked[:,:,:len(self.config.get('in_bands'))], stacked[:,:,len(self.config.get('in_bands')):]
    
    def get_dataset(self, glob):
        """Get the preprocessed training dataset
        Returns: 
        A tf.data.Dataset of training data.
        """
        glob = tf.compat.v1.io.gfile.glob(glob)
        
        dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
        dataset = dataset.map(self.parse_function, num_parallel_calls=5)
        
        return dataset
    
    
    def get_training_dataset(self):
        """Get the preprocessed training dataset
        Returns: 
        A tf.data.Dataset of training data.
        """
        glob = self.config.get('data_dir') + '/' + self.config.get('base_names')[0] + '*'
        dataset = self.get_dataset(glob)
        dataset = dataset.shuffle(self.config.get('shuffle_size')).batch(self.config.get('batch_size')).repeat()
        return dataset
    
    def get_validation_dataset(self):
        """Get the preprocessed validation dataset
        Returns: 
          A tf.data.Dataset of validation data.
        """
        glob = self.config.get('data_dir') + '/' + self.config.get('base_names')[1] + '*'
        dataset = self.get_dataset(glob)
        dataset = dataset.batch(1).repeat()
        return dataset

Writing AI_Platform/trainig_package/trainer/util.py


**Model**

In [5]:
!mkdir {ROOT_PATH+PACKAGE_FOLDER+'/models'}
!touch {ROOT_PATH+PACKAGE_FOLDER+'/models'}/__init__.py
!cp -r ../models/CNN {ROOT_PATH+PACKAGE_FOLDER+'/models'}/CNN
!cp -r ../models/MLP {ROOT_PATH+PACKAGE_FOLDER+'/models'}/MLP

In [6]:
%%writefile {ROOT_PATH+PACKAGE_FOLDER}/model.py

from google.cloud import storage
from google.cloud.storage import blob
import json

from .models.CNN.regression import deepvel as CNNregDeepVel, segnet as  CNNregSegNet, unet as CNNregUNet
from .models.CNN.segmentation import deepvel as CNNsegDeepVel, segnet as  CNNsegSegNet, unet as CNNsegUNet
from .models.MLP.regression import sequential1 as MLPregSequential1

def select_model(path):
    # Read training parameters from GCS
    client = storage.Client(project='skydipper-196010')
    bucket = client.get_bucket('geo-ai')
    blob = bucket.blob(path)
    config = json.loads(blob.download_as_string(client=client).decode('utf-8'))
    
    # Model's dictionary
    models = {'CNN':
              {
                  'regression': 
                  {
                      'deepvel': CNNregDeepVel.create_keras_model,
                      'segnet': CNNregSegNet.create_keras_model,
                      'unet': CNNregUNet.create_keras_model,
                  },
                  'segmentation': 
                  {
                      'deepvel': CNNsegDeepVel.create_keras_model,
                      'segnet': CNNsegSegNet.create_keras_model,
                      'unet': CNNsegUNet.create_keras_model,
                  }
              }, 
              'MLP': 
              {
                  'regression': 
                  {
                      'sequential1': MLPregSequential1.create_keras_model,
                  }
              }
             }
    
    return models.get(config.get('model_type')).get(config.get('model_output')).get(config.get('model_architecture'))

Writing AI_Platform/trainig_package/trainer/model.py


**Training task**

The following will create `task.py`, which will get the training and evaluation data, train the model and save it when it's done in a Cloud Storage bucket.

In [7]:
%%writefile {ROOT_PATH+PACKAGE_FOLDER}/task.py
"""Trains a Keras model"""

import os
import json
import argparse

import tensorflow as tf
from google.cloud import storage
from google.cloud.storage import blob

from .util import Util
from . import model

def get_args():
    """Argument parser.

    Returns:
      Dictionary of arguments.
    """
    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--params-file',
        type=str,
        required=True,
        help='GCS location where we have saved the training_params.json file')
    parser.add_argument(
        '--verbosity',
        choices=['DEBUG', 'ERROR', 'FATAL', 'INFO', 'WARN'],
        default='INFO')
    args, _ = parser.parse_known_args()
    return args

def train_and_validation(args):
    """Trains and evaluates the Keras model.

    Uses the Keras model defined in model.py and trains on data loaded and
    preprocessed in util.py. Saves the trained model in TensorFlow SavedModel
    format to the path defined in part by the --job-dir argument.
    """
    
    # Read training parameters from GCS
    client = storage.Client(project='skydipper-196010')
    bucket = client.get_bucket('geo-ai')
    blob = bucket.blob(args.params_file)
    config = json.loads(blob.download_as_string(client=client).decode('utf-8'))

    # Create the Keras Model
    selected_model = model.select_model(args.params_file)

    if not config.get('output_activation'):
        keras_model = selected_model(inputShape = (None, None, len(config.get('in_bands'))), nClasses = len(config.get('out_bands')))
    else:
        keras_model = selected_model(inputShape = (None, None, len(config.get('in_bands'))), nClasses = len(config.get('out_bands')), output_activation = config.get('output_activation'))

    # Compile Keras model
    optimizer = tf.keras.optimizers.Adam(lr=config.get('learning_rate'))
    keras_model.compile(loss=config.get('loss'), optimizer=optimizer, metrics=config.get('metrics'))


    # Pass a tfrecord
    util = Util(path = args.params_file) 
    training_dataset = util.get_training_dataset()
    validation_dataset = util.get_validation_dataset()

    # Setup TensorBoard callback.
    tensorboard_cb = tf.keras.callbacks.TensorBoard(os.path.join(config.get('job_dir'), 'logs'))

    # Train model
    keras_model.fit(
        x=training_dataset,
        steps_per_epoch=int(config.get('training_size') / config.get('batch_size')),
        epochs=config.get('epochs'),
        validation_data=validation_dataset,
        validation_steps=int(config.get('validation_size') / config.get('batch_size')),
        verbose=1,
        callbacks=[tensorboard_cb])

    tf.contrib.saved_model.save_keras_model(keras_model, os.path.join(config.get('job_dir'), 'model'))

if __name__ == '__main__':
    args = get_args()
    tf.logging.set_verbosity('INFO')
    train_and_validation(args)

Writing AI_Platform/trainig_package/trainer/task.py


**Create a .tar.gz distribution package**

In [8]:
def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))

In [9]:
output_filename = './AI_Platform/trainer-0.2.tar.gz'
source_dir = './AI_Platform/trainig_package/'

make_tarfile(output_filename, source_dir)

In [11]:
%env GOOGLE_APPLICATION_CREDENTIALS {env.privatekey_path}

env: GOOGLE_APPLICATION_CREDENTIALS=/Users/ikersanchez/Vizzuality/Keys/Skydipper/skydipper-196010-a4ce18e66917.json


In [13]:
client = storage.Client().from_service_account_json(env.privatekey_path)
bucket = client.get_bucket('geo-ai')
blob = bucket.blob('Train/trainer-0.2.tar.gz')
                     
blob.upload_from_filename(
    filename = output_filename, 
    content_type = 'text/plain',
    client=client
)
print(blob.public_url)

https://storage.googleapis.com/geo-ai/Train/trainer-0.2.tar.gz
